# Multivariate polynomials implementations

**Contributed by**: Benoît Legat

The SumOfSquares package is built on top of the [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl)
abstract interface. [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
is an implementation of this abstract interface so it can be used with
SumOfSquares. Moreover, any other implementation can be used as well. To
illustrate, we solve Examples 3.38 of [BPT12] with
[TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl),
another implementation of [MultivariatePolynomials](https://github.com/JuliaAlgebra/MultivariatePolynomials.jl).

[BPT12] Blekherman, G. & Parrilo, P. A. & Thomas, R. R.
*Semidefinite Optimization and Convex Algebraic Geometry*.
Society for Industrial and Applied Mathematics, **2012**.

In [1]:
import TypedPolynomials
TypedPolynomials.@polyvar x y
using SumOfSquares
import CSDP
model = SOSModel(CSDP.Optimizer)
con_ref = @constraint(model, 2x^4 + 5y^4 - x^2*y^2 >= -2(x^3*y + x + 1))
optimize!(model)
solution_summary(model)

CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.39e-01 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  9.2771541e+01 
Iter:  2 Ap: 1.00e+00 Pobj:  0.0000000e+00 Ad: 1.00e+00 Dobj:  5.3405022e+01 


* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.87271e-02


We see that the problem is feasible. The Sum-of-Squares decomposition can be
obtained as follows:

In [2]:
sos_decomposition(con_ref)

(-0.6797539388882385 - 0.09754482372919099*y - 0.4175945425725994*x + 2.1931519300021907*y^2 - 0.1269608752480834*x*y - 0.802586870770147*x^2)^2 + (0.9043585861943051 - 0.645497549386868*y + 0.27421735002417896*x - 0.10770166105288262*y^2 - 1.2293131412594318*x*y - 0.930017043547299*x^2)^2 + (-0.5739156167209172 - 1.385351263217042*y - 0.6219655290929103*x - 0.20409240982196936*y^2 - 0.1518243547747602*x*y + 0.4443814867923303*x^2)^2 + (0.5703017871356982 - 0.5184102408490358*y + 0.34367412466167074*x + 0.2555951335437903*y^2 + 0.7618448416641994*x*y - 0.020905670851165445*x^2)^2 + (0.0414029820781276 + 0.049110899408320925*y - 0.49990900774527297*x - 0.24792082949867678*y^2 + 0.29724484396646367*x*y - 0.5054176772471581*x^2)^2 + (-0.25245328219549723 - 0.06336616977802398*y + 0.25394133757830034*x - 0.10018803519680755*y^2 + 0.059602155796650916*x*y - 0.19381339537687217*x^2)^2

Why is there several implementations ?
Depending in the use-case, one implementation may be more appropriate than
another one. [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is faster than [DynamicPolynomials](https://github.com/JuliaAlgebra/DynamicPolynomials.jl/)
but it requires new compilation whenever the list of variables changes.
This means that [TypedPolynomials](https://github.com/JuliaAlgebra/TypedPolynomials.jl)
is not appropriate when the number of variables is dynamic or too large.
However, for a small number of variables, it can be faster.
When solving Sum-of-Squares programs, the time is mostly taken by the Semidefinite programming solver.
The time taken by SumOfSquares/JuMP/MathOptInterface are usually negligible
or it time is taken by manipulation of JuMP or MathOptInterface functions
therefore using TypedPolynomials over DynamicPolynomials may not make much difference in most cases.

One case for which using TypedPolynomials might be adequate is when
using domain defined by equalities (possibly also with inequalities).
Indeed, in that case, SumOfSquares computes the corresponding Gröbner basis which
may take a non-negligible amount of time for large systems of equalities.

To illustrate this, consider the computation of Gröbner basis for the
following system from [CLO05, p. 17].
The time taken by TypedPolynomials is below:

[CLO05] Cox, A. David & Little, John & O'Shea, Donal
*Using Algebraic Geometry*.
Graduate Texts in Mathematics, **2005**.
https://doi.org/10.1007/b138611

In [3]:
using BenchmarkTools
@btime let
    TypedPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  684.214 μs (6611 allocations: 1.34 MiB)


true

The time taken by DynamicPolynomials is as follows:

In [4]:
import DynamicPolynomials
@btime let
    DynamicPolynomials.@polyvar x y
    S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
    SemialgebraicSets.compute_gröbner_basis!(S.I)
end

  995.216 μs (9987 allocations: 1.26 MiB)


true

We see that TypedPolynomials is faster.
The time is still negligible for this small system but for larger systems, choosing TypedPolynomials may be helpful.
We can use this system in a Sum-of-Squares constraint as follows:

In [5]:
TypedPolynomials.@polyvar x y
S = @set x^3 * y + x == 2x^2 * y^2 && 3x^4 == y
poly = -6x - 4y^3 + 2x*y^2 + 6x^3 - 3y^4 + 13x^2 * y^2
model = Model(CSDP.Optimizer)
con_ref = @constraint(model, poly in SOSCone(), domain = S)
optimize!(model)
solution_summary(model)

Success: SDP solved
Primal objective value: 0.0000000e+00 
Dual objective value: 0.0000000e+00 
Relative primal infeasibility: 1.44e-16 
Relative dual infeasibility: 5.00e-11 
Real Relative Gap: 0.00e+00 
XZ Relative Gap: 3.04e-10 
DIMACS error measures: 1.79e-16 0.00e+00 5.00e-11 0.00e+00 0.00e+00 3.04e-10
CSDP 6.2.0
This is a pure primal feasibility problem.
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 7.64e-01 Pobj:  0.0000000e+00 Ad: 8.07e-01 Dobj:  2.6104524e+00 
Iter:  2 Ap: 7.71e-01 Pobj:  0.0000000e+00 Ad: 7.40e-01 Dobj: -5.9092654e-02 
Iter:  3 Ap: 8.08e-01 Pobj:  0.0000000e+00 Ad: 7.65e-01 Dobj:  5.5549062e-01 
Iter:  4 Ap: 7.56e-01 Pobj:  0.0000000e+00 Ad: 7.79e-01 Dobj:  4.8506840e-01 
Iter:  5 Ap: 6.95e-01 Pobj:  0.0000000e+00 Ad: 6.77e-01 Dobj:  4.3029645e-01 
Iter:  6 Ap: 6.35e-01 Pobj:  0.0000000e+00 Ad: 6.83e-01 Dobj:  2.2137529e-01 
Iter:  7 Ap: 6.21e-01 Pobj:  0.0000000e+00 Ad: 6.43e-01 Dobj:  1.2986447e-01 
Iter:  8 Ap: 

* Solver : CSDP

* Status
  Result count       : 1
  Termination status : OPTIMAL
  Message from the solver:
  "Problem solved to optimality."

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Objective value    : 0.00000e+00
  Dual objective value : 0.00000e+00

* Work counters
  Solve time (sec)   : 1.84589e-01


We obtain the following decomposition:

In [6]:
dec = sos_decomposition(con_ref, 1e-6)

(-6.84182741269261e-5 + 0.0006523762499047102*y - 0.00014179820285655732*x + 7.044759133483462*y^2 - 28.22908576786016*x*y + 25.36311206441547*x^2)^2 + (-1.4194958144543974e-5 + 0.0004276176113941165*y + 0.0008133609682906672*x + 22.478867347382973*y^2 - 13.864614651970733*x*y - 21.674926995277897*x^2)^2 + (-3.844627497275628e-5 - 0.0007282386100283765*y + 0.0005526974366361142*x + 0.12019410198039554*y^2 + 0.09016974474187112*x*y + 0.06697409634825505*x^2)^2 + (-6.124508498073891e-7 - 0.03996514972007246*y + 0.053405171968541894*x - 0.00026070294513364894*y^2 - 0.00019564294781865244*x*y - 0.0001440117861211223*x^2)^2

We can verify that it is correct as follows:

In [7]:
rem(dec - poly, S.I)

6.3610482265567761674591418480199933593866035153041593730449676513671875e-09 - 1.984495180153670773513842691689224313933710875109029121887989533253176783857045e-07y - 1.585727418775316157105128065897868229792668269230769230769230769230762428125679e-07x - 1.3043421811635876050861071462350082583725452423095703125e-05y² - 1.953301070667205763253804207124630920588970184326171875e-05xy - 7.30144730937999641984248455628403462469577789306640625e-06x² - 4.620111315745134561439044773578643798828125e-08y³ - 4.9722494122050875375862233340740203857421875e-08xy² - 4.1542469012887761441177314480910354177467525005340576171875e-08x²y + 6.495471237800442255460298978365384615384615384615384615384615384609942500553748e-09y⁴

Note that the difference between `dec` and `poly` is larger
than between the full gram matrix because `dec` is obtained by dropping
the lowest eigenvalues with the threshold `1e-6`; see `sos_decomposition`.

In [8]:
rem(gram_matrix(con_ref) - poly, S.I)

7.415334118042492821353990086440111095811289487755857408046722412109375e-09 + 1.794282224365083054771052935152158971149196403507047738784398788061192748573992e-08y + 3.381695265694326315916931399932274451622596153846153846153846153846156503436558e-09x + 7.41535164153726833458080136551870964467525482177734375e-09y² - 1.558649043165161174329114146530628204345703125e-14xy + 7.4153217138710203482077076841960661113262176513671875e-09x² - 7.01364892089865558470288912455240885416666666666666666666666666712726232293837e-13y³ + 1.530627476616549150397380193074544270833333333333333333333333333563631161469185e-13xy² - 3.91249532771809072073665447533130645751953125e-14x²y + 6.552970542263490362809254572941706730769230769230769230769230769230747972507539e-09y⁴

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*